# Keras Digit Recogniser

In [3]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from keras.models import Sequential
from keras.layers import Dense
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Flatten
from keras.layers.core import Activation
from keras.layers.core import Dropout
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping

In [4]:
df_data_1 = pd.read_csv('train.csv')
df_data_2 = pd.read_csv('test.csv')
train = df_data_1
test = df_data_2.values.astype('float32')

In [5]:
# preview data
train.head()
#test.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
labels = train.label.values.astype('int32')
train_img = train.loc[:,'pixel0':].values.astype('float32')
#labels
train_img.shape, test.shape
#type(labels)

((42000, 784), (28000, 784))

In [7]:
# convert labels to a categorical data type, as expected by keras model
labels = to_categorical(labels)
classes = labels.shape[1]
labels

array([[0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

In [8]:
# fix random seed for reproducibility
seed = 43
np.random.seed(seed)

## Multilayer Perceptron

Now we will build the neural network. We use a Keras Sequential object to build a simple feedforward network (multilayer perceptron, or MLP), with Relu input and hidden layer activations, and a softmax output layer with 10 classes. model.add() will be used to add layers.

In [9]:
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=784))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(10, activation='softmax'))

In [10]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
early_stopping = EarlyStopping(monitor='val_loss', patience=2)
model_fit = model.fit(train_img, labels, epochs=100, batch_size=64, validation_split=0.1, callbacks=[early_stopping])

Train on 37800 samples, validate on 4200 samples
Epoch 1/100
37800/37800 [==============================] - 3s 73us/step - loss: 0.7047 - acc: 0.7719 - val_loss: 0.3240 - val_acc: 0.9043
Epoch 2/100
37800/37800 [==============================] - 2s 50us/step - loss: 0.2758 - acc: 0.9200 - val_loss: 0.2895 - val_acc: 0.9202
Epoch 3/100
37800/37800 [==============================] - 2s 49us/step - loss: 0.2126 - acc: 0.9393 - val_loss: 0.2427 - val_acc: 0.9326
Epoch 4/100
37800/37800 [==============================] - 2s 49us/step - loss: 0.1882 - acc: 0.9466 - val_loss: 0.2128 - val_acc: 0.9419
Epoch 5/100
37800/37800 [==============================] - 2s 49us/step - loss: 0.1609 - acc: 0.9545 - val_loss: 0.2165 - val_acc: 0.9424
Epoch 6/100
37800/37800 [==============================] - 2s 50us/step - loss: 0.1505 - acc: 0.9576 - val_loss: 0.2043 - val_acc: 0.9462
Epoch 7/100
37800/37800 [==============================] - 2s 56us/step - loss: 0.1420 - acc: 0.9596 - val_loss: 0.1969 - v

In [12]:
pred = model.predict_classes(test)
pred, len(pred)

(array([2, 0, 9, ..., 3, 9, 2]), 28000)

In [13]:
result = pd.DataFrame({'ImageId':list(range(1,test.shape[0]+1)),
                       'Label':pred})
result.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3


In [15]:
result.to_csv('mlp_output.csv', index=False, header=True)

## Convolutional Neural Network

The MLP gives good accuracy, but is a generic model, not specialised for image data. Convolutional Neural Networks (CNNs) are specifically designed for processing grid-shaped data such as images, and should, theoretically, have much greater statistical efficiency when applied to this task, achieving better results.

In [16]:
cnn = Sequential()
cnn.add(Conv2D(4, (3,3), padding='same', input_shape=(28,28,1)))
cnn.add(Activation('relu'))
cnn.add(MaxPooling2D(pool_size=(2,2)))
cnn.add(Dropout(0.25))
cnn.add(Conv2D(8, (3,3), padding='same'))
cnn.add(Activation('relu'))
cnn.add(Conv2D(8, (3,3), padding='same'))
cnn.add(Activation('relu'))
cnn.add(MaxPooling2D(pool_size=(2,2)))
cnn.add(Dropout(0.25))
cnn.add(Flatten())
cnn.add(Dense(32, activation='relu'))
cnn.add(Dense(10, activation='softmax'))
cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

train_img_cnn = train_img.reshape(train_img.shape[0],28,28,1)

cnn_fit = cnn.fit(train_img_cnn, labels, epochs=100, batch_size=64, validation_split=0.1, callbacks=[early_stopping])

Train on 37800 samples, validate on 4200 samples
Epoch 1/100
37800/37800 [==============================] - 11s 299us/step - loss: 1.5244 - acc: 0.6075 - val_loss: 0.2908 - val_acc: 0.9114
Epoch 2/100
37800/37800 [==============================] - 11s 288us/step - loss: 0.3785 - acc: 0.8826 - val_loss: 0.1681 - val_acc: 0.9417
Epoch 3/100
37800/37800 [==============================] - 11s 304us/step - loss: 0.2569 - acc: 0.9178 - val_loss: 0.1334 - val_acc: 0.9562
Epoch 4/100
37800/37800 [==============================] - 11s 301us/step - loss: 0.2096 - acc: 0.9340 - val_loss: 0.1113 - val_acc: 0.9638
Epoch 5/100
37800/37800 [==============================] - 11s 301us/step - loss: 0.1817 - acc: 0.9416 - val_loss: 0.0914 - val_acc: 0.9695
Epoch 6/100
37800/37800 [==============================] - 12s 304us/step - loss: 0.1627 - acc: 0.9475 - val_loss: 0.0896 - val_acc: 0.9695
Epoch 7/100
37800/37800 [==============================] - 11s 303us/step - loss: 0.1419 - acc: 0.9541 - val_lo

In [17]:
pred = model.predict_classes(test)
result = pd.DataFrame({'ImageId':list(range(1,test.shape[0]+1)),
                       'Label':pred})
result.to_csv('cnn_output.csv', index=False, header=True)